Nenana Ice Classic: 

This project aims to predict what day Ice will melt on the Nenana river based on ice thickness readings. For more full documentation of the contest see https://www.nenanaakiceclassic.com/

In [1]:
import requests
import json
import pandas as pd

# The ice thickness data is stored in a public JSON format provided by weather.gov
# url example = "https://www.weather.gov/source/aprfc/it_2022.json"

# this is the variable to store station id, station name, measurement date, and ice thickness
# use this variable to create a data table
rows=[]


#This outer loop cycles through pervious years in a range determined by user
for year in range(1995, 2024):
    
    # URL is the JSON file. All the years are formatted identically the variable years allows the loop to be dynamic
    url = f'https://www.weather.gov/source/aprfc/it_{year}.json'

    response = requests.get(url)
    data = json.loads(response.text)
    
    #the JSON is formatted in a way that requires digging through some layers in order to access the required data
    for items in data:
        #print(items)
        for layer1 in data[items]:
            #print("###################")
            #print("  ",layer1)
            
        # for layer2 in data[items][layer1]:
        #    print("--------------------------------")
        #     print("  ",layer2)
                
            #The data is in the third level down, iterating through the second layer was not needed as it duplicated this data
            for layer3 in data[items][layer1]["features"]:

                #determine how many measurements were taken at each location
                numOfMeasurements = len(layer3['properties']['data'])


                date_and_measure_list = (layer3['properties']['data'])
                lid = layer3['properties']['lid']
                st_name = layer3['properties']['name']

                """print(lid)
                print(st_name)
                print(layer3['properties']['data'])
                print(numOfMeasurements)
                print(date_and_measure_list)

                print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                print("     ") """


                # this loop is used to add an additional row to the data if there were multiple measurements
                # at the same location. The way the data is formatted in the JSON there is only 1 entry for
                # locaiton details but multiple dates and measurements. Using this loop formats it in a way
                # that duplicates the location informaiton but adds new information in the date and measurement
                
                n = (numOfMeasurements)
                i=0
                while i < n:
                    rows.append([lid,st_name,date_and_measure_list[i][0],date_and_measure_list[i][1]])
                    i+=1
                    
#print(rows)                   
               


In [2]:
# This cell block uses pandas to convert the 'rows' variable into a Dataframe
df=pd.DataFrame(rows, columns=["Lid","Name","Date","Measurement"])
pd.set_option('display.width',1000)
print(df)

        Lid                             Name                 Date Measurement
0     ALSA2               Port Alsworth RAWS  1995-01-02 00:00:00          18
1     BTTA2         Koyukuk River nr Bettles  1994-12-31 00:00:00          24
2     RSHA2   Yukon River at Russian Mission  1994-12-31 00:00:00          17
3     SMLA2                   Smith Lake Ice  1994-12-24 00:00:00          16
4     SSLA2  Snowshoe Lake Ice near Nelchina  1995-01-02 00:00:00          36
...     ...                              ...                  ...         ...
4753  MXYA2    Kennicott River near McCarthy  2023-04-26 14:00:00          25
4754  MXYA2    Kennicott River near McCarthy  2023-04-27 14:00:00          24
4755  MXYA2    Kennicott River near McCarthy  2023-04-28 14:00:00          23
4756  MXYA2    Kennicott River near McCarthy  2023-04-29 14:00:00          22
4757  MXYA2    Kennicott River near McCarthy  2023-04-30 14:00:00          22

[4758 rows x 4 columns]


In [3]:
#Cell imports data to my google sheets for readability and other projects 
import gspread

# imports JSON credential key from local machine
gc = gspread.service_account(filename='C:/Github Repos/nenena-ice-4e058e270c26.json')
sh= gc.open("IceProject")

sh.sheet1.update('A1',rows)
Col_headers =['Station ID','Station Name','Reading Date', 'Ice Thickness']
sh.sheet1.insert_row(Col_headers, index=1)





{'spreadsheetId': '1mJ5Alr6Uut_1PmRoU2LqCR0h02DUbtNl4njcBCnEUs8',
 'updates': {'spreadsheetId': '1mJ5Alr6Uut_1PmRoU2LqCR0h02DUbtNl4njcBCnEUs8',
  'updatedRange': 'Sheet1!A1:D1',
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

In [18]:
#Create a new dataset that has the two closest points to the nenana river.
df_filter= (df[df["Lid"].isin(["NICA2","ENNA2"])])
df_filter

,Lid,Name,Date,Measurement
19,ENNA2,Tanana River at Nenana,1995-03-09 00:00:00,28
69,ENNA2,Tanana River at Nenana,1996-03-31 00:00:00,46
70,ENNA2,Tanana River at Nenana,1996-04-09 00:00:00,45
102,ENNA2,Tanana River at Nenana,1997-03-19 00:00:00,44
103,ENNA2,Tanana River at Nenana,1997-03-31 00:00:00,46
...,...,...,...,...
4733,NICA2,Nenana Ice,2023-03-21 22:00:00,32
4734,NICA2,Nenana Ice,2023-03-24 22:00:00,30
4735,NICA2,Nenana Ice,2023-03-27 22:00:00,26
4736,NICA2,Nenana Ice,2023-03-31 22:00:00,31
